In [1]:
import jieba
import numpy as np
import pandas as pd
import os
import re
from IPython.display import display

In [2]:
# 读取文档，并按行分隔
doc = open('HW3.txt', 'r', encoding='gbk').read().splitlines()
for sentence in doc[:10]:
    print(sentence)

基于用户声誉的鲁棒协同推荐算法
基于覆盖随机游走算法的服务质量预测
基于SOM神经网络的服务质量预测
异质信息网络中元路径感知的评分协同过滤
基于位置服务的分布式差分隐私推荐方法研究
基于时间访问轨迹的文件的智能推荐
协同过滤推荐系统中的用户博弈
基于网络表示学习的个性化商品推荐
人机协作的用户故事场景提取与迭代演进
基于图嵌入的软件项目源代码检索方法


In [3]:
# 添加自定义词库（可选）
for path in os.listdir('dictionary'):
    jieba.load_userdict('dictionary/' + path)

# 计算token，term数量
# token使用list存储，term使用set存储(可去重)
token, term = list(), set()
for sentence in doc:
    # 利用jieba进行分词 返回分词后的结果
    seg_list = list(jieba.cut(sentence, cut_all=False))
    # print('--------------------------------------------------------------')
    # print('sentence:', sentence)
    # print('seg_list:', seg_list)
    token.extend(seg_list)
    term.update(seg_list)
print('len(token) =', len(token))
print('len(term) =', len(term))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/m0/dsy0kdw13rb2g65q1x0bqn_00000gn/T/jieba.cache
Loading model cost 0.337 seconds.
Prefix dict has been built successfully.


len(token) = 6190
len(term) = 1128


In [4]:

hashtable = {}
for index, sentence in enumerate(doc):
    # 利用jieba进行分词 返回分词后的结果
    # 使用精确模式
    for word in jieba.cut(sentence, cut_all=False):
        if word in hashtable:
            hashtable[word].append(index+1)
            hashtable[word][0] += 1
        else:
            hashtable[word] = [1, index+1]
            
hashtable = dict(
    sorted(hashtable.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True))

# print(hashtable)


In [5]:
inverted_index = pd.DataFrame(columns=['term', 'doc.freq', 'postings list'])
for term in hashtable:
    inverted_index = inverted_index.append(
        {'term': term, 'doc.freq': hashtable[term][0], 'postings list': hashtable[term][1:]}, ignore_index=True)
display(inverted_index[:20])


,term,doc.freq,postings list
0,的,557,"[1, 2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 11, 12, 14,..."
1,基于,340,"[1, 2, 3, 5, 6, 8, 10, 12, 14, 15, 16, 17, 18,..."
2,推荐,255,"[1, 5, 6, 7, 8, 14, 18, 20, 22, 23, 24, 25, 26..."
3,学习,199,"[8, 29, 53, 54, 62, 137, 142, 151, 166, 167, 1..."
4,研究,196,"[5, 13, 20, 24, 29, 31, 32, 35, 40, 41, 42, 43..."
5,迁移,190,"[25, 246, 288, 364, 365, 366, 367, 368, 369, 3..."
6,方法,167,"[5, 10, 14, 19, 24, 26, 27, 33, 44, 64, 66, 67..."
7,算法,103,"[1, 2, 17, 18, 20, 21, 22, 23, 25, 34, 54, 59,..."
8,模型,93,"[18, 22, 23, 37, 45, 48, 50, 51, 52, 53, 57, 5..."
9,和,88,"[22, 25, 27, 34, 57, 80, 100, 105, 106, 107, 1..."


In [6]:
query_index = pd.DataFrame(columns=['term', 'doc.freq', 'postings list'])
query = ['迁移', '迁移学习', '推荐', '深度学习', '隐私', '跨领域', '跨域']
for term in query:
    if term in hashtable:
        query_index = query_index.append(
            {'term': term, 'doc.freq': hashtable[term][0], 'postings list': hashtable[term][1:]}, ignore_index=True)
    else:
        query_index = query_index.append(
            {'term': term, 'doc.freq': 0, 'postings list': []}, ignore_index=True)
display(query_index)


,term,doc.freq,postings list
0,迁移,190,"[25, 246, 288, 364, 365, 366, 367, 368, 369, 3..."
1,迁移学习,0,[]
2,推荐,255,"[1, 5, 6, 7, 8, 14, 18, 20, 22, 23, 24, 25, 26..."
3,深度学习,0,[]
4,隐私,11,"[5, 14, 93, 96, 128, 140, 149, 255, 319, 350, ..."
5,跨领域,0,[]
6,跨域,27,"[216, 369, 370, 371, 377, 387, 398, 406, 407, ..."


In [7]:
# # encoding=utf-8
# import jieba

# # jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持
# strs=["我来到北京清华大学","乒乓球拍卖完了","中国科学技术大学"]
# for str in strs:
#     seg_list = jieba.cut(str,use_paddle=True) # 使用paddle模式
#     print("Paddle Mode: " + '/'.join(list(seg_list)))

# seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
# print("Full Mode: " + "/ ".join(seg_list))  # 全模式

# seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
# print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

# seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
# print(", ".join(seg_list))

# seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
# print(", ".join(seg_list))

In [8]:
# import fool

# text = "一个傻子在北京"
# print(fool.cut(text))
# # ['一个', '傻子', '在', '北京']

In [9]:
# from LAC import LAC

# # 装载分词模型
# lac = LAC(mode='seg')

# # 单个样本输入，输入为Unicode编码的字符串
# text = u"LAC是个优秀的分词工具"
# seg_result = lac.run(text)

# # 批量样本输入, 输入为多个句子组成的list，平均速率会更快
# texts = [u"LAC是个优秀的分词工具", u"百度是一家高科技公司"]
# seg_result = lac.run(texts)

In [10]:
import jieba

if __name__ == '__main__':
    jieba.load_userdict('jieba_dict.txt')
    document = dict()
    i = 1
    with open("HW3.txt", "r", encoding='gbk') as f:  # 打开文件,必须用txt的编码格式，否则会报错
        for line in f.readlines():
            line = line.strip('。')  # 删去标点符号
            line = line.strip('，')
            line = line.strip('\n')
            line = line.strip('——')
            document[i] = jieba.lcut(line, cut_all=True)  # 规格化为小写
            i = i + 1
    # print(data)
    all_words = []
    for i in document.values():
        all_words.extend(i)
    words = set(all_words)
    words.remove('')
    # print(set_all_words)

    invert_index = dict()
    for b in words:
        temp = []
        for j in document.keys():
            field = document[j]
            if b in field:  # 查找单词在文章里面是否存在
                temp.append(j)
        invert_index[b] = temp
    # print(invert_index)
    print('迁移: '+str(invert_index['迁移']))
    print('迁移学习: '+str(invert_index['迁移学习']))
    print('推荐: '+str(invert_index['推荐']))
    print('深度学习: '+str(invert_index['深度学习']))
    print('隐私: '+str(invert_index['隐私']))
    # print('跨领域: ' + str(invert_index['跨领域']))
    print('跨域: ' + str(invert_index['跨域']))



FileNotFoundError: [Errno 2] No such file or directory: 'jieba_dict.txt'